In [286]:
# Throughput 
# Graph plot change 
# Halls theorem : it states that if there is a set S having n elements and a set T having m elements with the preference for each i in S as {T2,T3,T4},{T1,T3,T4}....{T1,Tm}
# then if for each subset in S, P if cardinality(P) <= cardinality(Neighbour(P)) then a matching is possible for this bipartite graph (no two edges must have a common vertex)
# A -> (1, 2) , B -> (2), C -> (3)
# task -> frequency of arrival, size, CPU cycles required, deadline
# ith[1,0,0,0,0]
# [0,1,0,0,0]
# 0.5
# 0th [ 0 0 0 0 0 0.4 0 0 0 0 ]
# 1th [ 0 0.2 0 0 0 0 0 0 0 0 ]

# matrix of offloading where Aij represents the fraction of bits offloaded to edge server j of task i
# A mobile device Ui can only offload to a single edge server Ej
# the local computation time = ((1 - sum(Aij)) * Qi * Ci) / fi (Qi -> total bits, Ci -> CPU cycles for single bit, fi -> CPU cycles/sec for the mobile device)
# local energy consumption = (1 - sum(Aij)) * Qi * Ci * k * (fi)^2 (k -> mobile constant)
# Uplink Rate = Wlog(1 + (Pi * Gij) / N) (W -> uplink channel bandwidth, Pi -> transmission power of device i, Gij -> channel gain, N -> Gaussian Noise)
# Upload Time = sum((Aij * Qi) / Rij)
# Time at MEC = sum((Aij * Qi * Ci) / fj)
# Energy consumed in Transmission = sum(((Aij * Qi) / Rij) * Pi)

# Overall Delay = T(local) + T(transmission) + T(MEC)
# Overall Energy = E(local) + E(transmission)
# utility
# Priority of task = Total CPU Cycles / Execution Deadline
# Priority of Edge Server = Fj / (dist(u,Mj)) ^ x

# Threshold for edge server selection priority >= 0.6
# (A,B,C), (A,D), (A,E) -> 0.6
# (A,D,E), (A,E), (A,W) -> 0.7

# Threshold vs Energy, Threshold vs Time, Threshold vs Matching Found

# Assumption
# Each mobile device has a single task to perform

# threshold, offloading factor

In [287]:
# ni(t) = number of computing task at the ith cluster in tth time slot
# distance b/w cluster of edge server and mobile devices

# [[1,2], [2,3], [3,4]]

# C([[1,2],[3,4]]) <=  C([1,2,3,4])

In [288]:
%store -r mobile_constant
%store -r  channel_gain
%store -r gaussian_noise
%store -r channel_bandwidth
%store -r mobile_device_x_coordinates
%store -r mobile_device_y_coordinates
%store -r edge_device_cluster_x_coordinates
%store -r edge_device_cluster_y_coordinates
%store -r mobile_device_computing_power
%store -r mobile_device_transmission_power
%store -r task_bits
%store -r CPU_cycle_task_bit
%store -r execution_deadline
%store -r edge_device_computing_power
%store -r free_cluster_next_time

In [289]:
%run constants.ipynb


Stored 'mobile_constant' (int)
Stored 'channel_gain' (float)
Stored 'gaussian_noise' (float)
Stored 'channel_bandwidth' (float)
Stored 'mobile_device_x_coordinates' (list)
Stored 'mobile_device_y_coordinates' (list)
Stored 'edge_device_cluster_x_coordinates' (list)
Stored 'edge_device_cluster_y_coordinates' (list)
Stored 'mobile_device_computing_power' (list)
Stored 'mobile_device_transmission_power' (list)
Stored 'task_bits' (list)
Stored 'CPU_cycle_task_bit' (list)
Stored 'execution_deadline' (list)
Stored 'edge_device_computing_power' (list)
Stored 'free_cluster_next_time' (list)


In [290]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random

In [291]:
# [10, 20, 10, 30, 40, 20, 10, 5, 30, 20]
# 5
# [[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 0], [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1], [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 2], [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3], [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 4]]
# 10
# 0
# 10
def cardinality_of_neighbours(nums):
    # print(nums)
    # [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1]
    distinct_elements = set()
    for i in range(0,len(nums),1):
        # print(len(nums[i][0]))
        for j in range(0,len(nums[i][0]),1):
            # print(j ,"A")
            distinct_elements.add(nums[i][0][j])
    return len(distinct_elements)

In [292]:
def halls_theorem(configuration, subset, index):
  if index == len(configuration):
    if len(subset) <= cardinality_of_neighbours(subset):
      return True
    else:
      return False
  ans = True
  # including
  subset.append(configuration[index])
  ans = ans and halls_theorem(configuration, subset, index + 1)
  subset.pop()
  # excluding
  ans = ans and halls_theorem(configuration, subset, index + 1)
  return ans

In [293]:
def selection_algorithm(pair_possible_configuration_task, visited, index, matching): # O(n * 2 ^ (m))
    if index == len(pair_possible_configuration_task):
        return True
    found = False
    for edge_server in pair_possible_configuration_task[index][0]:
        if edge_server not in visited:
            visited.add(edge_server)
            matching.append([pair_possible_configuration_task[index][1],edge_server])
            found = found or selection_algorithm(pair_possible_configuration_task, visited, index + 1, matching)
            if found:
                return True
            matching.pop()
            visited.remove(edge_server)
    return found

In [294]:
def edge_device_selection(edge_server_x_coordinates, edge_server_y_coordinates,mobile_device_x_coordinates,mobile_device_y_coordinates,threshold,offloading_factor_value,mobile_device_computing_power,mobile_device_transmission_power,task_bits,CPU_cycle_task_bit,execution_deadline,edge_device_computing_power):
    total_time_taken = 0

    number_of_mobile_devices = len(mobile_device_x_coordinates)
    number_of_edge_servers = len(edge_server_x_coordinates)
    # print(mobile_device_computing_power)
    # print(number_of_mobile_devices)
    possible_configuration = []
    for i in range(0, number_of_mobile_devices):
      possible_edge_servers = []
      for j in range(0, number_of_edge_servers):
        planar_distance = math.sqrt((edge_server_x_coordinates[j]-mobile_device_x_coordinates[i])**2 + (edge_server_y_coordinates[j]-mobile_device_y_coordinates[i])**2)
        if(planar_distance == 0):
          planar_distance = 1.0
        # print(planar_distance)
        priority = mobile_device_computing_power[i] / planar_distance
        if priority >= threshold:
          possible_edge_servers.append([j,priority])  
      sorted(possible_edge_servers, key=lambda edge_server: edge_server[1], reverse = True)
      sorted_by_priority_edge_server = []
      for list in possible_edge_servers:
        sorted_by_priority_edge_server.append(list[0])
      possible_configuration.append(sorted_by_priority_edge_server)  
      

    # pair of configuration for a md and index and sorted in priority of task
    pair_possible_configuration_task = []
    
    priority_task = []
    for i in range(0,number_of_mobile_devices):
      priority_task.append([(task_bits[i] * CPU_cycle_task_bit[i]) / execution_deadline[i],i])
    sorted(priority_task, key=lambda priority: priority[0], reverse = True)

    # [[edge servers], mobile_device]
    for i in range(0,len(priority_task)):
      index = priority_task[i][1]
      pair_possible_configuration_task.append([possible_configuration[index],index])

    # [[configuration for ith task],i]
    current_excluded = []
    while len(pair_possible_configuration_task)>0:
      # send possible configuration enough for hall's theorem
      if halls_theorem(pair_possible_configuration_task, [], 0):
        # print("This configuration follows halls theorem")
        matching = []
        visited = set()
        selection_algorithm(pair_possible_configuration_task,visited,0,matching)
        local_computation_time = 0
        local_energy_consumption = 0
        upload_time = 0
        edge_device_computing_time = 0
        transmission_energy = 0

        for i in range(0, len(pair_possible_configuration_task)):
          offloaded_bits = offloading_factor_value
          selected_edge_device = 0

          for j in range(0,len(matching)):
            if (matching[j][0] == pair_possible_configuration_task[i][1]):
              selected_edge_device = matching[j][1]
          
          uplink_rate = channel_bandwidth * math.log(1 + (mobile_device_computing_power[pair_possible_configuration_task[i][1]]*channel_gain) / gaussian_noise)
          
          local_computation_time += ((1 - offloaded_bits) * task_bits[pair_possible_configuration_task[i][1]] * CPU_cycle_task_bit[pair_possible_configuration_task[i][1]]) / mobile_device_computing_power[pair_possible_configuration_task[i][1]]
          local_energy_consumption += (1 - offloaded_bits) * task_bits[pair_possible_configuration_task[i][1]] * CPU_cycle_task_bit[pair_possible_configuration_task[i][1]] * (mobile_device_computing_power[pair_possible_configuration_task[i][1]])**2
          upload_time += (offloaded_bits * task_bits[pair_possible_configuration_task[i][1]]) / uplink_rate
          
          # print(pair_possible_configuration_task[i][1], "A")
          
          edge_device_computing_time += (offloaded_bits * task_bits[pair_possible_configuration_task[i][1]] * CPU_cycle_task_bit[pair_possible_configuration_task[i][1]]) / edge_device_computing_power[selected_edge_device]
          transmission_energy += ((offloaded_bits * task_bits[pair_possible_configuration_task[i][1]]) / uplink_rate) * mobile_device_transmission_power[pair_possible_configuration_task[i][1]]  
        
        total_time_taken += local_computation_time + edge_device_computing_time + upload_time
        pair_possible_configuration_task = current_excluded
        current_excluded = []

      else:
        current_excluded.insert(0,pair_possible_configuration_task[len(pair_possible_configuration_task)-1])
        # [[edge_server], mobile_device]
        pair_possible_configuration_task.pop()
    
    return total_time_taken


    